<a href="https://colab.research.google.com/github/Joykareko/Data-Science-Projects/blob/main/Parkinson's_Disease_Detection_ML_Project_15.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Parkinson's is a nervous system disorder and is progressive in nature.

We will be using SVM in this model

In [31]:
#Importing dependencies/libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn import svm

In [32]:
#loading the data
park_data = pd.read_csv('/content/parkinsons.csv')
park_data.head(3)

,name,MDVP:Fo(Hz),MDVP:Fhi(Hz),MDVP:Flo(Hz),MDVP:Jitter(%),MDVP:Jitter(Abs),MDVP:RAP,MDVP:PPQ,Jitter:DDP,MDVP:Shimmer,MDVP:Shimmer(dB),Shimmer:APQ3,Shimmer:APQ5,MDVP:APQ,Shimmer:DDA,NHR,HNR,status,RPDE,DFA,spread1,spread2,D2,PPE
0,phon_R01_S01_1,119.992,157.302,74.997,0.00784,0.00007,0.00370,0.00554,0.01109,0.04374,0.426,0.02182,0.03130,0.02971,0.06545,0.02211,21.033,1,0.414783,0.815285,-4.813031,0.266482,2.301442,0.284654
1,phon_R01_S01_2,122.400,148.650,113.819,0.00968,0.00008,0.00465,0.00696,0.01394,0.06134,0.626,0.03134,0.04518,0.04368,0.09403,0.01929,19.085,1,0.458359,0.819521,-4.075192,0.335590,2.486855,0.368674
2,phon_R01_S01_3,116.682,131.111,111.555,0.01050,0.00009,0.00544,0.00781,0.01633,0.05233,0.482,0.02757,0.03858,0.03590,0.08270,0.01309,20.651,1,0.429895,0.825288,-4.443179,0.311173,2.342259,0.332634


In [33]:
park_data.shape

(195, 24)

In [34]:
park_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195 entries, 0 to 194
Data columns (total 24 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   name              195 non-null    object 
 1   MDVP:Fo(Hz)       195 non-null    float64
 2   MDVP:Fhi(Hz)      195 non-null    float64
 3   MDVP:Flo(Hz)      195 non-null    float64
 4   MDVP:Jitter(%)    195 non-null    float64
 5   MDVP:Jitter(Abs)  195 non-null    float64
 6   MDVP:RAP          195 non-null    float64
 7   MDVP:PPQ          195 non-null    float64
 8   Jitter:DDP        195 non-null    float64
 9   MDVP:Shimmer      195 non-null    float64
 10  MDVP:Shimmer(dB)  195 non-null    float64
 11  Shimmer:APQ3      195 non-null    float64
 12  Shimmer:APQ5      195 non-null    float64
 13  MDVP:APQ          195 non-null    float64
 14  Shimmer:DDA       195 non-null    float64
 15  NHR               195 non-null    float64
 16  HNR               195 non-null    float64
 1

In [35]:
#checking for missing values
park_data.isnull().sum()

name                0
MDVP:Fo(Hz)         0
MDVP:Fhi(Hz)        0
MDVP:Flo(Hz)        0
MDVP:Jitter(%)      0
MDVP:Jitter(Abs)    0
MDVP:RAP            0
MDVP:PPQ            0
Jitter:DDP          0
MDVP:Shimmer        0
MDVP:Shimmer(dB)    0
Shimmer:APQ3        0
Shimmer:APQ5        0
MDVP:APQ            0
Shimmer:DDA         0
NHR                 0
HNR                 0
status              0
RPDE                0
DFA                 0
spread1             0
spread2             0
D2                  0
PPE                 0
dtype: int64

In [36]:
#statistical measures
park_data.describe()

,MDVP:Fo(Hz),MDVP:Fhi(Hz),MDVP:Flo(Hz),MDVP:Jitter(%),MDVP:Jitter(Abs),MDVP:RAP,MDVP:PPQ,Jitter:DDP,MDVP:Shimmer,MDVP:Shimmer(dB),Shimmer:APQ3,Shimmer:APQ5,MDVP:APQ,Shimmer:DDA,NHR,HNR,status,RPDE,DFA,spread1,spread2,D2,PPE
count,195.000000,195.000000,195.000000,195.000000,195.000000,195.000000,195.000000,195.000000,195.000000,195.000000,195.000000,195.000000,195.000000,195.000000,195.000000,195.000000,195.000000,195.000000,195.000000,195.000000,195.000000,195.000000,195.000000
mean,154.228641,197.104918,116.324631,0.006220,0.000044,0.003306,0.003446,0.009920,0.029709,0.282251,0.015664,0.017878,0.024081,0.046993,0.024847,21.885974,0.753846,0.498536,0.718099,-5.684397,0.226510,2.381826,0.206552
std,41.390065,91.491548,43.521413,0.004848,0.000035,0.002968,0.002759,0.008903,0.018857,0.194877,0.010153,0.012024,0.016947,0.030459,0.040418,4.425764,0.431878,0.103942,0.055336,1.090208,0.083406,0.382799,0.090119
min,88.333000,102.145000,65.476000,0.001680,0.000007,0.000680,0.000920,0.002040,0.009540,0.085000,0.004550,0.005700,0.007190,0.013640,0.000650,8.441000,0.000000,0.256570,0.574282,-7.964984,0.006274,1.423287,0.044539
25%,117.572000,134.862500,84.291000,0.003460,0.000020,0.001660,0.001860,0.004985,0.016505,0.148500,0.008245,0.009580,0.013080,0.024735,0.005925,19.198000,1.000000,0.421306,0.674758,-6.450096,0.174351,2.099125,0.137451
50%,148.790000,175.829000,104.315000,0.004940,0.000030,0.002500,0.002690,0.007490,0.022970,0.221000,0.012790,0.013470,0.018260,0.038360,0.011660,22.085000,1.000000,0.495954,0.722254,-5.720868,0.218885,2.361532,0.194052
75%,182.769000,224.205500,140.018500,0.007365,0.000060,0.003835,0.003955,0.011505,0.037885,0.350000,0.020265,0.022380,0.029400,0.060795,0.025640,25.075500,1.000000,0.587562,0.761881,-5.046192,0.279234,2.636456,0.252980
max,260.105000,592.030000,239.170000,0.033160,0.000260,0.021440,0.019580,0.064330,0.119080,1.302000,0.056470,0.079400,0.137780,0.169420,0.314820,33.047000,1.000000,0.685151,0.825288,-2.434031,0.450493,3.671155,0.527367


In [37]:
#distribution of target variable
#1 means positive presence of Parkinsons while 0 means negative
park_data['status'].value_counts()

1    147
0     48
Name: status, dtype: int64

In [38]:
park_data.groupby('status').mean()

,MDVP:Fo(Hz),MDVP:Fhi(Hz),MDVP:Flo(Hz),MDVP:Jitter(%),MDVP:Jitter(Abs),MDVP:RAP,MDVP:PPQ,Jitter:DDP,MDVP:Shimmer,MDVP:Shimmer(dB),Shimmer:APQ3,Shimmer:APQ5,MDVP:APQ,Shimmer:DDA,NHR,HNR,RPDE,DFA,spread1,spread2,D2,PPE
status,,,,,,,,,,,,,,,,,,,,,,
0,181.937771,223.636750,145.207292,0.003866,0.000023,0.001925,0.002056,0.005776,0.017615,0.162958,0.009504,0.010509,0.013305,0.028511,0.011483,24.678750,0.442552,0.695716,-6.759264,0.160292,2.154491,0.123017
1,145.180762,188.441463,106.893558,0.006989,0.000051,0.003757,0.003900,0.011273,0.033658,0.321204,0.017676,0.020285,0.027600,0.053027,0.029211,20.974048,0.516816,0.725408,-5.333420,0.248133,2.456058,0.233828


Data preprocessing

In [39]:
X = park_data.drop(columns = ['name','status'], axis =1)
Y = park_data['status']

In [40]:
print(X.shape,Y.shape)

(195, 22) (195,)


In [41]:
#splitting the data to training and testing data
X_train,X_test,Y_train,Y_test = train_test_split(X,Y, test_size = 0.2, stratify = Y ,random_state = 2)
print(X_train.shape,X_test.shape,Y_train.shape,Y_test.shape)

(156, 22) (39, 22) (156,) (39,)


Standardizing the data

In [42]:
scaler = StandardScaler()
scaler.fit(X_train)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [43]:
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [44]:
print(X_train)

[[-3.37789850e-01 -1.86151275e-01 -9.11085922e-01 ...  3.02808525e-01
   3.67380761e-01 -1.01626972e-01]
 [ 1.09942206e+00  2.52399879e-01  7.59431971e-01 ...  9.62684763e-01
   2.30410182e-01  7.25430092e-03]
 [-8.75220075e-01 -5.64868721e-01 -3.69947894e-01 ... -1.24083946e-03
  -1.27562573e+00 -5.03037967e-01]
 ...
 [ 9.67834202e-01  1.38914623e-01 -8.24451036e-01 ...  5.83176337e-01
   5.94403638e-01 -2.56870663e-01]
 [-7.69983726e-01 -6.17537239e-01 -4.08691589e-01 ...  2.01206260e-01
  -9.18334164e-01 -4.43401072e-01]
 [ 1.19847659e+00  4.93351249e-01  1.11785168e+00 ... -1.03979251e-01
   5.12603529e-01 -5.39510027e-01]]


In [45]:
print(X_test)

[[ 6.75664548e-01  4.30692802e-02  9.17000470e-01 -4.65043840e-01
  -7.49745604e-01 -3.93228335e-01 -3.77439566e-01 -3.94525767e-01
   4.45619551e-01  2.19598173e-01  5.21466719e-01  5.50343466e-01
   2.33273889e-01  5.21470425e-01  1.16130564e-01 -1.29432625e+00
   8.21532645e-01  1.19747938e-01  1.25597046e-02  1.02371713e+00
   1.83847640e+00 -9.57975551e-02]
 [ 2.14053706e+00  5.75095585e-01  2.69990518e+00 -7.34959454e-01
  -1.06696913e+00 -6.19284025e-01 -6.30896620e-01 -6.20586823e-01
  -6.53557602e-01 -6.64300480e-01 -5.71749737e-01 -6.30067211e-01
  -6.80568916e-01 -5.72107026e-01 -4.63775964e-01  2.99892220e-01
  -4.08092287e-01 -1.53468433e+00 -1.02431660e+00 -8.41894550e-01
  -3.00324761e-01 -1.00212990e+00]
 [-1.38183818e+00 -9.55917721e-01 -7.26885474e-01  1.56841734e-01
   8.36372029e-01  2.13175024e-01  2.32999252e-01  2.14284168e-01
   5.35991668e-01  3.84392837e-01  6.79342665e-01  5.16269755e-01
   2.32711870e-01  6.79681691e-01 -1.47813603e-01 -6.35730329e-01
   4.4

In [46]:
#making the model
classifier = svm.SVC()
classifier.fit(X_train,Y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [47]:
#making predictions
train_pred = classifier.predict(X_train)
score_1 = accuracy_score(Y_train,train_pred)
print('Accuracy score on training data:',score_1)

Accuracy score on training data: 0.8846153846153846


In [48]:
#test data
test_data = classifier.predict(X_test)
score_2 = accuracy_score(Y_test,test_data)
print('Accuracy score on training data:',score_2)

Accuracy score on training data: 0.8717948717948718


In [49]:
#Making a prediction system
input_data = [119.99200,157.30200,74.99700,0.00784,0.00007,0.00370,0.00554,0.01109,0.04374,0.42600,0.02182,0.03130,0.02971,0.06545,0.02211,21.03300,0.414783,0.815285,-4.813031,0.266482,2.301442,0.284654]

#changing to numpy array
input_data_as_np = np.asarray(input_data)

#reshaping
input_data_as_np_reshaped = input_data_as_np.reshape(1,-1)

#standardize the data
input_data_standard = scaler.transform(input_data_as_np_reshaped)

#fitting the data to the model
prediction = classifier.predict(input_data_standard)
if (prediction[0]==1):
  print(prediction,'Person has Parkinsons')
else:
  print(prediction,'Person is Parkinson Free')

[1] Person has Parkinsons


In [50]:
#Making a prediction system
input_data = [214.28900,260.27700,77.97300,0.00567,0.00003,0.00295,0.00317,0.00885,0.01884,0.19000,0.01026,0.01161,0.01373,0.03078,0.04398,21.20900,0.462803,0.664357,-5.724056,0.190667,2.555477,0.148569]

#changing to numpy array
input_data_as_np = np.asarray(input_data)

#reshaping
input_data_as_np_reshaped = input_data_as_np.reshape(1,-1)

#standardize the data
input_data_standard = scaler.transform(input_data_as_np_reshaped)

#fitting the data to the model
prediction = classifier.predict(input_data_standard)
if (prediction[0]==1):
  print(prediction,'Person has Parkinsons')
else:
  print(prediction,'Person is Parkinson Free')

[1] Person has Parkinsons
